In [ ]:
from PIL import Image
import fitz
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import pipeline

In [3]:
# --- PDF Text Extraction ---
def extract_text_with_ocr(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        text += pytesseract.image_to_string(img, lang='ben')
    return text
    
pdf_path = "HSC26-Bangla1st-Paper.pdf"
text = extract_text_with_ocr(pdf_path)
print(f"Extracted text (first 500 chars): {text[:500]}")

Extracted text (first 500 chars): বাংলা
১ম পত্র

আলোচ্য বিষয়

অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,

কল করো ৩ 16910
 নিয়বিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।

তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।

তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।

' তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্টয সম্পর্কে জ্ঞানলাভ করবে।

নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে
পারবে।

' মানুষ আশা নিয়ে বেঁচে থাকে অনুপমের দু


In [4]:
# --- Chunking ---
def chunk_text(text, chunk_size=200):
    sentences = text.split("।")
    chunks = []
    chunk = ""
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        if len(chunk) + len(sentence) < chunk_size:
            chunk += sentence + "।"
        else:
            chunks.append(chunk.strip())
            chunk = sentence + "।"
    if chunk:
        chunks.append(chunk.strip())
    return chunks


chunks = chunk_text(text)
for i, chunk in enumerate(chunks[:10]):
    print(f"Chunk {i}: {chunk}")
    

Chunk 0: বাংলা
১ম পত্র

আলোচ্য বিষয়

অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,

কল করো ৩ 16910
 নিয়বিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।
Chunk 1: তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।' তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্টয সম্পর্কে জ্ঞানলাভ করবে।
Chunk 2: নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে
পারবে।
Chunk 3: ' মানুষ আশা নিয়ে বেঁচে থাকে অনুপমের দুষ্টান্তে মানবর্জীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে
জ্ঞানলাভ করবে।১।অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?

ক) ডাক্তারি 'খ) ওকালতি গ) মাস্টারি ঘ) ব্যবসা
২।
Chunk 4: মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-
ক) প্রতিপত্তি খ) প্রভাব গ) বিচক্ষণতা ঘ) কূটবুদ্ধি

'নিচের অনুচ্ছেদটি পড়ে ৩ ও ৪ সংখ্যক প্রশ্নের উত্তর দাও।'পিতুহীন দীপুর চাচাই ছিলেন পরিবারের কর্তা।
Chunk 5: দীপু শিক্ষিত হলেও তার সিদ্ধান্ত নেওয়ার ক্ষমতা ছিল না।চাচা
'তার বিয়ের উদ্যোগ নিলেও যৌতুক নিয়ে

In [5]:
# --- Retrieval ---
def retrieve(query, model, index, chunks, top_k=1):  # Only top chunk for context
    query_emb = model.encode([query])
    D, I = index.search(np.array(query_emb), top_k)
    return [chunks[i] for i in I[0]]
    
# --- Answer Generation (Hugging Face Transformers) ---
# Use a multilingual QA model since Bangla-specific QA models are not available
qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-multilingual-cased",
    tokenizer="distilbert-base-multilingual-cased"
)

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

# --- Embedding & Vector DB ---
def embed_chunks(chunks, model):
    embeddings = model.encode(chunks)
    return np.array(embeddings)
    
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = embed_chunks(chunks, embed_model)
index = build_faiss_index(embeddings)
queries = [
        "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
        "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?",
        "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?",
        "Who is called a true gentleman according to Anupam?",
        "Who is mentioned as Anupam's luck god?",
        "What was Kalyani's real age at the time of marriage?"
    ]
for q in queries:
    retrieved = retrieve(q, embed_model, index, chunks, top_k=3)
    context = " ".join(retrieved)
    answer = qa_pipeline(question=q, context=context)['answer']
    print(f"Q: {q}\nA: {answer}\n")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Q: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
A: অনুপম সেই সীমাবদ্ধতা

Q: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
A: উদ্দীপকের বরের

Q: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
A: উদ্দীপকের বরের

Q: Who is called a true gentleman according to Anupam?
A: স্পষ্টবাদী ও ব্যক্তিত্ববান

Q: Who is mentioned as Anupam's luck god?
A: অনুপমের মামার দেওয়া বিলাতি

Q: What was Kalyani's real age at the time of marriage?
A: প্রাতিষ্ঠানিক শিক্ষাগ্রহণে নিরুৎসাহী

